In [1]:
from sqlalchemy import create_engine
from dotenv import load_dotenv
from llm.azure_llm import create_azure_chat_llm
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
import pandas as pd
import time

load_dotenv()

True

In [2]:
# Tabela 1
df_name_1 = "Chart of AccountsV0 copy.xlsx"
df_1 = pd.read_excel(f"inputs/Databases/{df_name_1}", sheet_name="Assets")
df_1.head()

,Assets,Category,Sub Category,Group Account Number
0,NON CURRENT,INTANGIBLE ASSETS,GOODWILL,10101
1,NON CURRENT,INTANGIBLE ASSETS,GOODWILL,10102
2,NON CURRENT,INTANGIBLE ASSETS,GOODWILL,10103
3,NON CURRENT,INTANGIBLE ASSETS,GOODWILL,10111
4,NON CURRENT,INTANGIBLE ASSETS,GOODWILL,10112


In [3]:
# Tabela 1.2
df_name_12 = "Chart of AccountsV0 copy.xlsx"
df_12 = pd.read_excel(f"inputs/Databases/{df_name_12}", sheet_name="Liabilities")
df_12.head()

,Liabilities,Category,Sub Category,Group Account Number
0,EQUITY,STOCKHOLDERS EQUITY / RESERVES,SHARE CAPITAL,20101
1,EQUITY,STOCKHOLDERS EQUITY / RESERVES,SHARE CAPITAL,20102
2,EQUITY,STOCKHOLDERS EQUITY / RESERVES,SHARE CAPITAL,20103
3,EQUITY,STOCKHOLDERS EQUITY / RESERVES,ADDITIONAL PAID IN CAPITAL,20201
4,EQUITY,STOCKHOLDERS EQUITY / RESERVES,ADDITIONAL PAID IN CAPITAL,20216


In [4]:
df333 = pd.merge(df_1, df_12, on="Group Account Number", how="left")

In [5]:
# Tabela 1.3
df_name_13 = "Chart of AccountsV0 copy.xlsx"
df_13 = pd.read_excel(f"inputs/Databases/{df_name_13}", sheet_name="P&L Accounts")
df_13.head()

,Measure,Category,Sub Category,Group Account Number
0,SALES,SALES,TRADE SALES,40101
1,SALES,SALES,TRADE SALES,40102
2,SALES,SALES,TRADE SALES,40103
3,SALES,SALES,TRADE SALES,40105
4,SALES,SALES,TRADE SALES,40111


In [6]:
# Tabela 1.4
df_name_14 = "Chart of AccountsV0 copy.xlsx"
df_14 = pd.read_excel(f"inputs/Databases/{df_name_14}", sheet_name="Companies")
df_14.head()

,Code,Country,Company Name
0,83,Brazil,83 - XYZ Ltda. - Brazil
1,98,Argentina,98 - XYZ Ltda. - Argentina
2,303,Mexico,303 - XYZ Ltda. - Mexico
3,340,Chile,340 - XYZ Ltda. - Chile
4,341,Colombia,341 - XYZ Ltda. - Colombia


In [7]:
# Tabela 2
df_name_2 = "Master Data.xlsx"
df_2 = pd.read_excel(f"inputs/Databases/{df_name_2}")
df_2.head()

,Company Code,Currency Type,Fiscal Year,Posting Period,Group Account Number,Account Number,Value,Currency,Ledger
0,83,10,2024,10,19110,19110041,5581.82,BRL,0L
1,83,10,2024,10,19120,19120044,322413.34,BRL,0L
2,83,10,2024,10,19120,19120081,1278401.63,BRL,0L
3,83,10,2024,10,19110,19110044,20485.10,BRL,0L
4,83,10,2024,10,19120,19120084,1336394.27,BRL,0L


In [8]:
df_2.loc[df_2["Account Number"]=="43106200"].sort_values(by="Value", ascending=False)

,Company Code,Currency Type,Fiscal Year,Posting Period,Group Account Number,Account Number,Value,Currency,Ledger
204772,341,10,2023,12,43106,43106200,4.392441e+11,COP,0L
204370,341,10,2023,16,43106,43106200,4.392441e+11,COP,0L
308912,341,10,2023,16,43106,43106200,4.392441e+11,COP,T1
309343,341,10,2023,12,43106,43106200,4.392441e+11,COP,T1
228125,341,10,2023,12,43106,43106200,4.392441e+11,COP,L1
...,...,...,...,...,...,...,...,...,...
55098,437,10,2024,1,43106,43106200,2.960216e+06,PEN,0L
190826,437,60,2024,1,43106,43106200,2.960216e+06,PEN,T1
151606,437,60,2024,1,43106,43106200,2.960216e+06,PEN,0L
59452,437,10,2024,1,43106,43106200,2.960216e+06,PEN,T1


In [9]:
# Tabelas
tables = {"assets":df_1, "liabilites":df_12, "pl_accounts":df_13, "companies":df_14, "master_data":df_2}

In [10]:
for table in tables.values():
    for col in table.columns:
        print(col)
    print("\n")

Assets
Category
Sub Category
Group Account Number


Liabilities
Category
Sub Category
Group Account Number


Measure
Category
Sub Category
Group Account Number


Code
Country
Company Name


Company Code
Currency Type
Fiscal Year
Posting Period
Group Account Number
Account Number
Value
Currency
Ledger




#### WCM

In [11]:
table_names = ["Trades Receivable","Trades Payable", "Working Capital"]
tables = {}
for tab in table_names:
    df= pd.read_excel("inputs/Databases/WCM Datav0.xlsx", sheet_name=tab)
    tables[tab] = df

In [17]:
tables["Trades Receivable"].head()

,Month Year,Country,Customer,Trades Receivable,Overdue,DSO,Sales,CEI,ART,Due Interval
0,2025-01-01,Brasil,Rios Siqueira Ltda.,523204,NaN,74,626818,0.700970,0.865261,Not Due
1,2025-01-01,Argentina,Pires Albuquerque Ltda.,645416,NaN,44,449996,0.482591,0.374159,1-90 Days
2,2025-01-01,Colombia,Mendes da Cruz Ltda.,916597,NaN,68,140790,0.786232,0.207056,91-180 Days
3,2025-01-01,Peru,Silveira Ferreira Ltda.,849506,NaN,121,657611,0.607331,0.666727,181-270 Days
4,2025-01-01,Chile,Siqueira Abreu Ltda.,862145,NaN,116,403406,0.346957,0.787860,270-360 Days


In [25]:
tables["Trades Receivable"]["Country"].unique()

array(['Brasil', 'Argentina', 'Colombia', 'Peru', 'Chile', 'Mexico',
       'Sudão', 'Cazaquistão', 'Tailândia', 'Bósnia e Herzegovina',
       'Belize', 'Bolívia', 'Alemanha', 'Ilha de Man', 'Madagascar',
       'Argélia', 'Territórios Austrais Franceses', 'Guiné', 'Ilha Wake',
       'Iêmen', 'Sérvia', 'União Europeia', 'Índia', 'Fiji', 'Samoa',
       'Eritreia', 'Canadá', 'Moldávia', 'Vietnam', 'Butão',
       'República Checa', 'Congo-Kinshasa', 'Jibuti', 'Vaticano',
       'Bahamas', 'Suíça', 'Faroé'], dtype=object)

In [18]:
tables["Trades Payable"].head()

,Month Year,Country,Vendors,Trades Payable,Overdue,DPO,TR/ICO to pay,Due Interval
0,2025-01-01,Brasil,Moura Pires Ltda.,380769,NaN,156,0.259593,Not Due
1,2025-01-01,Argentina,Albuquerque Ramos Ltda.,25103,NaN,140,0.383037,1-90 Days
2,2025-01-01,Colombia,Sales Cirino Ltda.,529888,NaN,12,0.427473,91-180 Days
3,2025-01-01,Peru,Moreira Jesus Ltda.,298890,NaN,60,0.782684,181-270 Days
4,2025-01-01,Chile,Costa Leão Ltda.,892674,NaN,58,0.842521,270-360 Days


In [19]:
tables["Working Capital"].head()

,Month Year,Country,Working Capital,Due Interval
0,2025-01-01,Brasil,142435,Overdue
1,2025-01-01,Argentina,620313,1-90 Days
2,2025-01-01,Colombia,386709,91-180 Days
3,2025-01-01,Peru,550616,181-270 Days
4,2025-01-01,Chile,-30529,270-360 Days


In [13]:
# Bancos de dados
db_name = "db/acelerador-analytics-finance.db"
engine = create_engine(f"sqlite:///{db_name}")

In [14]:
# # Adicionando tabelas ao db
# for df_name in tables.keys():
#     df = tables[df_name]
#     for col in df.columns:
#         if "nnamed" in col:
#             df= df.drop(col, axis=1)
#     df.to_sql(df_name, engine, index=False)

In [15]:
db = SQLDatabase(engine)

In [16]:
db.get_usable_table_names()

['Trades Payable',
 'Trades Receivable',
 'Working Capital',
 'assets',
 'companies',
 'liabilites',
 'master_data',
 'pl_accounts']